In [1]:
import os
import pandas as pd 
import numpy as np
from multiprocessing import Pool 
import multiprocessing
from data_loader import data_loader #data_loader.py 파일을 다운 받아 주셔야 합니다. 
from tqdm import tqdm
from functools import partial

import matplotlib.pyplot as plt
from collections import Counter

import glob


In [2]:
def data_loader_all(func, path, train, nrows, **kwargs):
    '''
    Parameters:
    
    func: 하나의 csv파일을 읽는 함수 
    path: [str] train용 또는 test용 csv 파일들이 저장되어 있는 폴더 
    train: [boolean] train용 파일들 불러올 시 True, 아니면 False
    nrows: [int] csv 파일에서 불러올 상위 n개의 row 
    lookup_table: [pd.DataFrame] train_label.csv 파일을 저장한 변수 
    event_time: [int] 상태_B 발생 시간 
    normal: [int] 상태_A의 라벨
    
    Return:
    
    combined_df: 병합된 train 또는 test data
    '''
    
    # 읽어올 파일들만 경로 저장 해놓기 
    files_in_dir = os.listdir(path)
    
    files_path = [path+'/'+file for file in files_in_dir]
    
    if train :
        func_fixed = partial(func, nrows = nrows, train = True, lookup_table = kwargs['lookup_table'], event_time = kwargs['event_time'], normal = kwargs['normal'])
        
    else : 
        func_fixed = partial(func, nrows = nrows, train = False)
    
    
    # 여러개의 코어를 활용하여 데이터 읽기 
    if __name__ == '__main__':sns.distplot(test.iloc[:,:10])
        pool = Pool(processes = multiprocessing.cpu_count()) 
        df_list = list(tqdm(pool.imap(func_fixed, files_path), total = len(files_path)))
        pool.close()
        pool.join()
    
    # 데이터 병합하기 
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df
    

In [3]:
# # nrows를 50으로 진행 600하면 터짐. 높이면 성능 올라갈듯. 제너레이터 필요.

# train_path = 'train'
# test_path = 'test'
# label = pd.read_csv('train_label.csv')
# train = data_loader_all(data_loader, path = train_path, train = True, nrows = 600, normal = 999, event_time = 0, lookup_table = label)

100%|██████████| 827/827 [01:34<00:00,  8.73it/s]


In [4]:
# train.to_csv('./all_train.csv', index=False)

In [ ]:
train = pd.read_csv('./all_train.csv')

train_label = train.label
train_id = train.id
train_time = train.time

train = train.drop(['id','time','label'], axis=1)

In [ ]:
train_path = 'train'
test_path = 'test'
label = pd.read_csv('train_label.csv')
test = data_loader_all(data_loader, path = test_path, train = False, nrows = 60)


In [ ]:
train.shape, test.shape

In [ ]:
train = pd.concat([train,test],axis=0)


In [ ]:
train.shape

## minmaxscaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
train = minmax.fit_transform(train)

In [ ]:
train.head()

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2000)

In [ ]:
train = pca.fit_transform(train)

In [ ]:
#라벨분포. 라벨 분포에 따라 적은 라벨은 학습이 덜될수도 있을듯?
#업샘플링, 다운샘플링 연구 필요
# plt.hist(label.label, bins=50)

## 모델링

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_jobs=-1)


In [12]:
#학습

rfc.fit(train[:],train_label)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# #Fitting the PCA algorithm with our Data
# #Plotting the Cumulative Summation of the Explained Variance
# plt.figure()
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('Number of Components')
# plt.ylabel('Variance (%)') #for each component
# plt.title('Pulsar Dataset Explained Variance')
# plt.show()

## RF 학습

In [ ]:
minmax_self = MinMaxScaler() # csv 별 self normailze시 사용

In [13]:
# test csv 들을 넣어서 각csv의 row별로 라벨 별 확률값인 predict_proba를 평균내서 각 csv별 predict_proba를 구함.
# 1154, 1168은 문자열포함이라 추론에러남. 그냥 빼고 진행. 나중에 0으로 채움.

results = []

for i in range(828,1548):
    print(i)
    if i != 1154 and i != 1168:
        tmp = pd.read_csv('./test/'+str(i)+'.csv')
        tmp = tmp.drop(['time'], axis=1)
        tmp = minmax.transform(tmp) #self normalize 할시 minmax_self.fit_transform으로 변경
        tmp = pca.transform(tmp) # pca pc 개수 변경시 컬럼수 조정 필요.
        result = rfc.predict_proba(tmp)

        results.append(np.mean(result,axis=0))

828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
10

In [ ]:
#위에서 뺀 1154, 1168자리에 0으로 채움

results.insert(326,[0]*198)
results.insert(340,[0]*198)

In [ ]:
#id열 추가하고 DF만듬.

id = pd.DataFrame(np.arange(828,1548), columns=['id'])
submit = pd.DataFrame(results)

submit = pd.concat([id,submit], axis=1)

In [ ]:
submit.head()

In [ ]:
submit.shape


In [ ]:
#id 1154에 0들어갔는지 확인

submit[submit['id']==1154]

In [ ]:
#id 1154에 0들어갔는지 확인

submit[submit['id']==1168]

In [ ]:
#데이터프레임 생성

submit.to_csv('./results/submit_.csv', index=False)

# 아래ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ실험ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ


## 보스톤 집값 실험

In [3]:
from sklearn.datasets import load_boston

boston = load_boston()
dir(boston)

['DESCR', 'data', 'feature_names', 'filename', 'target']

In [4]:
dfX = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])

In [9]:
dfX.shape, dfy.shape

((506, 13), (506, 1))

In [11]:
train_X = dfX[:400] 
train_y = dfy[:400]

test_X = dfX[400:]
test_y = dfy[400:]

In [13]:
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((400, 13), (400, 1), (106, 13), (106, 1))

In [30]:
#한번에 다 학습

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rfr = RandomForestRegressor()

rfr.fit(train_X, train_y)

prediction = rfr.predict(test_X)

mean_squared_error(test_y, prediction)

/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


18.309730754716966

In [47]:
#나누어 학습

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rfr = RandomForestRegressor()

rfr.fit(train_X[:100], train_y[:100])
rfr.fit(train_X[100:200], train_y[100:200])
rfr.fit(train_X[200:300], train_y[200:300])
rfr.fit(train_X[300:400], train_y[300:400])

prediction = rfr.predict(test_X)

mean_squared_error(test_y, prediction)

/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


16.578427858490553

## 유방암 분류 실험


In [11]:
from sklearn.datasets import load_breast_cancer
breast = load_breast_cancer()


In [12]:
dfX = pd.DataFrame(breast.data, columns=breast.feature_names)
dfy = pd.DataFrame(breast.target, columns=["MEDV"])

In [8]:
dfX.shape, dfy.shape

((178, 13), (178, 1))

In [9]:
train_X = dfX[:400] 
train_y = dfy[:400]

test_X = dfX[400:]
test_y = dfy[400:]

In [10]:
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((178, 13), (178, 1), (0, 13), (0, 1))

In [73]:
#한번에 다 학습

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
rfr = RandomForestClassifier()

rfr.fit(train_X, train_y)

prediction = rfr.predict(test_X)

rfr.score(test_X, test_y)

/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


0.9644970414201184

In [89]:
#나누어 학습

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
rfr = RandomForestClassifier()

rfr.fit(train_X[:100], train_y[:100])
rfr.fit(train_X[100:200], train_y[100:200])
rfr.fit(train_X[200:300], train_y[200:300])
rfr.fit(train_X[300:400], train_y[300:400])

prediction = rfr.predict(test_X)

rfr.score(test_X, test_y)

/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/home/seo/anaconda3/envs/dacon/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()

0.9644970414201184